# Fetching feature view

In [ ]:
import hopsworks
from datetime import datetime

In [ ]:
project = hopsworks.login()

In [ ]:
fs = project.get_feature_store()

In [ ]:
fv = fs.get_feature_view(name="fraud_feature_view", version=1)

# Materialize all the logs in the feature view and fetch loggin feature groups

In [ ]:
# Materialize logs written into Kafka.
fv.materialize_log(wait=True)

In [ ]:
# Fetch feature logger
feature_logger = fv.feature_logging

In [ ]:
# Get logging feature groups
feature_group_transformed, feature_group_untransfromed = feature_logger.transformed_features, feature_logger.untransformed_features 

# Fetch feature groups and Preparing Data

In [ ]:
# Fetch feature groups
fg_transactions = fs.get_feature_group(name="transactions_feature_group", version = 1) 
fg_credit_cards = fs.get_feature_group(name="credit_cards_feature_group", version = 1)  

In [ ]:
# Featching features required in the feature group
transaction_features = [feature.name for feature in fg_transactions.features]
credit_card_features = [feature.name for feature in fg_credit_cards.features]

# Add prefix to credit card features since they have been prefixed in the feature view.
prefixed_credit_card_features = ["fg2_" + feature if feature != "cc_num" else feature for feature in credit_card_features]

In [ ]:
# Read untransformed data from logging feature group
logging_dataframe = feature_group_untransfromed.read()

In [ ]:
# Extract features required to be inserted into feature groups from logging features dataframe
transactions_dataframe = logging_dataframe[transaction_features]
credit_card_dataframe = logging_dataframe[prefixed_credit_card_features]

# Renaming creadit cards dataframe to have name required for the feature group
credit_card_dataframe = credit_card_dataframe.rename(columns={prefixed_name:required_name for prefixed_name, required_name in zip(prefixed_credit_card_features, credit_card_features)})

# Backfilling Data

In [ ]:
# Here `transform` is set to `False` so that on-demand features are not computed on ingestion as they are already computed and present in the dataframe.
fg_transactions.insert(transactions_dataframe, transform=False)
fg_credit_cards.insert(credit_card_dataframe, transform=False)